In [1]:
import aocd
from dotenv import load_dotenv
import networkx as nx

load_dotenv()

puzzle = aocd.get_puzzle(day=24, year=2024)
text = puzzle.input_data
examples = puzzle.examples
print(examples)

lines = text.splitlines()

example = examples[0].input_data.splitlines()

examples

[Example(input_data='x00: 1\nx01: 1\nx02: 1\ny00: 0\ny01: 1\ny02: 0\n\nx00 AND y00 -> z00\nx01 XOR y01 -> z01\nx02 OR y02 -> z02', answer_a='z', answer_b='aaa,aoc,bbb,ccc,eee,ooo,z24,z99', extra=None)]


[Example(input_data='x00: 1\nx01: 1\nx02: 1\ny00: 0\ny01: 1\ny02: 0\n\nx00 AND y00 -> z00\nx01 XOR y01 -> z01\nx02 OR y02 -> z02', answer_a='z', answer_b='aaa,aoc,bbb,ccc,eee,ooo,z24,z99', extra=None)]

In [3]:
text = examples[0].input_data
lines = text.splitlines()

example="""x00: 0
x01: 1
x02: 0
x03: 1
x04: 0
x05: 1
y00: 0
y01: 0
y02: 1
y03: 1
y04: 0
y05: 1

x00 AND y00 -> z05
x01 AND y01 -> z02
x02 AND y02 -> z01
x03 AND y03 -> z03
x04 AND y04 -> z04
x05 AND y05 -> z00"""


text = example

In [18]:
from functools import cache
# full backtracking
g = nx.DiGraph()
init, graph_lines = map(lambda t: t.splitlines(), text.split("\n\n"))
N = 45


def make_graph_tuple(graph_lines):
    g = nx.DiGraph()
    ops = {}
    for line in graph_lines:
        a, op, b, _, c = line.split()
        g.add_edge(a, c)
        g.add_edge(b, c)
        ops[c] = op

    return g, ops, list(nx.topological_sort(g))


def solve(g, val, ops, order):
    for n in order:
        if n in val:
            continue
        p1, p2 = g.predecessors(n)
        op = ops[n]
        if op == "XOR":
            val[n] = val[p1] ^ val[p2]
        if op == "AND":
            val[n] = val[p1] & val[p2]
        if op == "OR":
            val[n] = val[p1] | val[p2]


def add(x, y, g, ops, order):
    assert len(x) == len(y)

    val = {}
    for i, (xx, yy) in enumerate(zip(reversed(x), reversed(y))):
        val[f"x{i:02}"], val[f"y{i:02}"] = int(xx), int(yy)

    solve(g, val, ops, order)
    zs = sorted([(k, v) for k, v in val.items() if k.startswith("z")], reverse=True)
    return "".join(str(v) for _, v in zs)


def all_lines_leading_to(h, node, graph_lines):
    ancestors = nx.ancestors(h, node)
    ancestors.add(node)
    return [line for line in graph_lines if line.split()[-1] in ancestors]

def switch(graph_lines, a,b):
    i1, i2 = graph_lines.index(a), graph_lines.index(b)
    h_lines = list(graph_lines)
    a1, a2 = a.split("->")
    b1, b2 = b.split("->")
    n1 = f"{a1}->{b2}"
    n2 = f"{b1}->{a2}"
    h_lines[i1], h_lines[i2] = n1, n2
    return h_lines


def apply_switches(graph_lines, switches):
    h_lines = list(graph_lines)
    try:
        for a, b in switches:
            h_lines = switch(h_lines, a, b)
    except ValueError:
        print(switches)
        raise  ValueError
    return h_lines

@cache
def _nb_errors(lines):
    errors = 0
    try:
        graph_tuple = make_graph_tuple(lines)
    except nx.NetworkXUnfeasible:
        return 1000
    
    xs, ys = f"{0:045b}", f"{0:045b}"
    if int(add(xs, ys, *graph_tuple), 2) != 0:
        errors += 1

    for i in range(N):
        possibilites = [
            (0, 2**i),
            (2**i, 0),
            (2**i-1, 2**i- 1),
            (2**i, 2**i),
        ]
        for x, y in possibilites:
            xs, ys = f"{x:045b}", f"{y:045b}"
            if int(add(xs, ys, *graph_tuple), 2) != x + y:
                errors += 1
        
    return errors
def nb_errors(lines):
    return _nb_errors(tuple(lines))

def backtrack(switches, lines, current_error):
    if len(switches) == 4:
        if nb_errors(lines) == 0:
            return switches
        return 
    

    already_switched = set(tt for t in switches for tt in t)

    for a in lines:
        for b in lines:
            if a == b:
                continue
            if a in already_switched or b in already_switched:
                continue

            s_prime = switches + [(a, b)]
            h_lines = apply_switches(lines, s_prime)
            e = nb_errors(h_lines)
            if e < current_error:
                print(e, "Switching", a, b, s_prime)
                res = backtrack(s_prime, lines, e)
                if res:
                    return res
        

    return None

e = nb_errors(graph_lines)

print("Starting with", e)
switches = backtrack([], graph_lines, nb_errors(graph_lines))



Starting with 38
33 Switching dfb XOR bfn -> hbk sjr OR tck -> z14 [('dfb XOR bfn -> hbk', 'sjr OR tck -> z14')]
30 Switching cjb XOR srm -> z19 y18 AND x18 -> z18 [('dfb XOR bfn -> hbk', 'sjr OR tck -> z14'), ('cjb XOR srm -> z19', 'y18 AND x18 -> z18')]
29 Switching dvw AND rpg -> z23 rjm XOR gjr -> z24 [('dfb XOR bfn -> hbk', 'sjr OR tck -> z14'), ('cjb XOR srm -> z19', 'y18 AND x18 -> z18'), ('dvw AND rpg -> z23', 'rjm XOR gjr -> z24')]
27 Switching gtm AND rmt -> mkv x34 XOR y34 -> tfn [('dfb XOR bfn -> hbk', 'sjr OR tck -> z14'), ('cjb XOR srm -> z19', 'y18 AND x18 -> z18'), ('dvw AND rpg -> z23', 'rjm XOR gjr -> z24'), ('gtm AND rmt -> mkv', 'x34 XOR y34 -> tfn')]
28 Switching cht OR mkv -> mqf x34 XOR y34 -> tfn [('dfb XOR bfn -> hbk', 'sjr OR tck -> z14'), ('cjb XOR srm -> z19', 'y18 AND x18 -> z18'), ('dvw AND rpg -> z23', 'rjm XOR gjr -> z24'), ('cht OR mkv -> mqf', 'x34 XOR y34 -> tfn')]
26 Switching y34 AND x34 -> cvh x34 XOR y34 -> tfn [('dfb XOR bfn -> hbk', 'sjr OR tck 

KeyboardInterrupt: 

In [9]:
g = nx.DiGraph()
init, graph_lines = map(lambda t: t.splitlines(), text.split("\n\n"))
N = 45


def make_graph_tuple(graph_lines):
    g = nx.DiGraph()
    ops = {}
    for line in graph_lines:
        a, op, b, _, c = line.split()
        g.add_edge(a, c)
        g.add_edge(b, c)
        ops[c] = op

    return g, ops, list(nx.topological_sort(g))


def solve(g, val, ops, order):
    for n in order:
        if n in val:
            continue
        p1, p2 = g.predecessors(n)
        op = ops[n]
        if op == "XOR":
            val[n] = val[p1] ^ val[p2]
        if op == "AND":
            val[n] = val[p1] & val[p2]
        if op == "OR":
            val[n] = val[p1] | val[p2]


def add(x, y, g, ops, order):
    assert len(x) == len(y)

    val = {}
    for i, (xx, yy) in enumerate(zip(reversed(x), reversed(y))):
        val[f"x{i:02}"], val[f"y{i:02}"] = int(xx), int(yy)

    solve(g, val, ops, order)
    zs = sorted([(k, v) for k, v in val.items() if k.startswith("z")], reverse=True)
    return "".join(str(v) for _, v in zs)


def all_lines_leading_to(h, node, graph_lines):
    ancestors = nx.ancestors(h, node)
    ancestors.add(node)
    return [line for line in graph_lines if line.split()[-1] in ancestors]


def eval(lines, i):
    m1 = i - 1
    if m1 < 0:
        return True
    possibilites = [
        (0, 0),
        (0, 2**m1),
        (2**m1, 0),
        (2**m1-1, 2**m1- 1),
    ]
    try:
        graph_tuple = make_graph_tuple(lines)
    except nx.NetworkXUnfeasible:
        return False

    for x, y in possibilites:
        xs, ys = f"{x:045b}", f"{y:045b}"

        if int(add(xs, ys, *graph_tuple), 2) != x + y:
            return False
    return True


valid_lines = []
dirty_lines = list(graph_lines)
badsies = []

for i in range(N):
    z = f"z{i:02}"
    print(z)

    g, ops, order = make_graph_tuple(graph_lines)

    if not eval(graph_lines, i):
        # candidate_brokens = all_lines_leading_to(g, z, graph_lines)
        candidate_brokens = list(dirty_lines)
        candidate_brokens = [c for c in candidate_brokens if c not in valid_lines]
        try:
            for a in candidate_brokens:
                for b in dirty_lines:
                    if a == b:
                        continue

                    i1, i2 = graph_lines.index(a), graph_lines.index(b)
                    h_lines = list(graph_lines)
                    a1, a2 = a.split("->")
                    b1, b2 = b.split("->")
                    n1 = f"{a1}->{b2}"
                    n2 = f"{b1}->{a2}"
                    h_lines[i1], h_lines[i2] = n1, n2

                    if eval(h_lines, i) and all(eval(h_lines, j) for j in range(i)):
                        graph_lines = h_lines
                        dirty_lines.remove(a)
                        dirty_lines.remove(b)
                        valid_lines.append(n1)
                        valid_lines.append(n2)
                        badsies.extend([a2.strip(), b2.strip()])
                        print("Swapping", a, b, badsies)
                        raise ValueError
            print("Not working")

        except ValueError:
            continue

badsies1 = badsies
res = ",".join(sorted(badsies))
print(res)


z00
z01
z02
z03
z04
z05
z06
z07
z08
z09
z10
z11
z12
z13
z14
z15


KeyboardInterrupt: 

In [14]:
graph_lines_1 = graph_lines

In [12]:
g = nx.DiGraph()

init, graph_lines = map(lambda t: t.splitlines(), text.split("\n\n"))
# assert graph_lines != graph_lines_base
N = 45
# N = 6

def eval(g, i, ops, verbose=False):
    possibilities = [(0, 0, 0), (1, 0, 1), (0, 1, 1), (1, 1, 0)]
    try:
        order = list(nx.topological_sort(g))
    except nx.NetworkXUnfeasible:
        return False
    for x, y, z in possibilities:
        val = {}
        for j in range(N):
            val[f"x{j:02d}"] = int(j == i) * x
            val[f"y{j:02d}"] = int(j == i) * y
        try:
            solve(g, val,ops, order)
        except ValueError:
            return False

        zs = sorted([(k, v) for k, v in val.items() if k.startswith("z")], reverse=True)
        zs = "".join(str(v) for _, v in zs[-(i + 1) :])

        expected = str(z) + "0" * (i)
        if zs != expected:
            if not verbose:
                return False

            print(zs, expected)
            xs = sorted(
                [(k, v) for k, v in val.items() if k.startswith("x")], reverse=True
            )
            print("".join(str(v) for v in xs))

            ys = sorted(
                [(k, v) for k, v in val.items() if k.startswith("y")], reverse=True
            )
            print("".join(str(v) for _, v in ys))

            zs = sorted(
                [(k, v) for k, v in val.items() if k.startswith("z")], reverse=True
            )
            print("".join(str(v) for _, v in zs))

            print()
            return False

    return True

def eval_plus_one(g, i, ops, verbose=False):
    try:
        order = list(nx.topological_sort(g))
    except nx.NetworkXUnfeasible:
        return False
    
    val = {}
    for j in range(N):
        val[f"x{j:02d}"] = int(j == i)
        val[f"y{j:02d}"] = int(j == i)

    try:
        solve(g, val, ops, order)
    except ValueError:
        return False

    zs = sorted([(k, v) for k, v in val.items() if k.startswith("z")], reverse=True)
    zs = "".join(str(v) for _, v in zs[-(i + 2) :])

    expected = "1" + "0" * (i+1)
    if zs != expected:
        if not verbose:
            return False

        print(zs, expected)
        xs = sorted(
            [(k, v) for k, v in val.items() if k.startswith("x")], reverse=True
        )
        print("".join(str(v) for v in xs))

        ys = sorted(
            [(k, v) for k, v in val.items() if k.startswith("y")], reverse=True
        )
        print("".join(str(v) for _, v in ys))

        zs = sorted(
            [(k, v) for k, v in val.items() if k.startswith("z")], reverse=True
        )
        print("".join(str(v) for _, v in zs))

        print()
        return False

    return True



valid_lines = []
dirty_lines = list(graph_lines)
badsies = []

for i in range(N):
    z = f"z{i:02}"
    g, ops, _ = make_graph_tuple(graph_lines)
    print(z)

    if not eval(g, i, ops):
        candidate_brokens = all_lines_leading_to(g, z, graph_lines)
        # candidate_brokens = list(dirty_lines)
        candidate_brokens = [c for c in candidate_brokens if c not in valid_lines]
        try:
            for a in candidate_brokens:
                for b in dirty_lines:
                    if a == b:
                        continue

                    i1, i2 = graph_lines.index(a), graph_lines.index(b)
                    h_lines = list(graph_lines)
                    a1, a2 = a.split("->")
                    b1, b2 = b.split("->")

                    n1 = f"{a1}->{b2}"
                    n2 = f"{b1}->{a2}"
                    h_lines[i1], h_lines[i2] = n1, n2
                    try:
                        h, ops, _ = make_graph_tuple(h_lines)
                    except nx.NetworkXUnfeasible:
                        continue

                    if eval(h, i, ops):
                        if all(eval(h, j, ops) for j in range(i)):
                            print("Swapping", a, b)
                            graph_lines = h_lines
                            g = h
                            dirty_lines.remove(a)
                            dirty_lines.remove(b)
                            valid_lines.append(n1)
                            valid_lines.append(n2)
                            badsies.extend([a2.strip(), b2.strip()])
                            raise ValueError
            print("Not working")

        except ValueError:
            continue

print(",".join(sorted(badsies)))

z00
z01
z02
z03
z04
z05
z06
z07
z08
z09
z10
z11
z12
z13
z14
Swapping y14 XOR x14 -> dfb y14 AND x14 -> tck
z15
z16
z17
z18
Swapping y18 AND x18 -> z18 x18 XOR y18 -> grp
z19
z20
z21
z22
z23
Swapping dvw AND rpg -> z23 dvw XOR rpg -> dbb
z24
z25
z26
z27
z28
z29
z30
z31
z32
z33
z34
Swapping y34 AND x34 -> cvh x34 XOR y34 -> tfn
z35
z36
z37
z38
z39
z40
z41
z42
z43
z44
cvh,dbb,dfb,grp,tck,tfn,z18,z23


In [50]:
graph_tuple = make_graph_tuple(graph_lines)

x,y = 10000000,10000000010

xs, ys = f"{x:045b}", f"{y:045b}"
xs = "100000000000000000000000000001001011010000000"
ys = "100000000000000000000000000011110010000001010"

x = int(xs, 2)
y = int(ys, 2)

res = add(xs, ys, *graph_tuple)
res_int = int(res, 2)

res, res_int, x + y, res_int == x + y


('1000000000000000000000000000100111101010001010',
 35184372251274,
 35184372251274,
 True)

In [ ]:
res = ",".join(sorted(badsies+ badsies1))
res

In [ ]:
s = "cvh,dbb,dfb,grp,tck,tfn,z18,z23"
aocd.submit(s, part=2, day=24, year=2024)

# part 1

In [ ]:
text = examples[0].input_data
lines = text.splitlines()

example="""x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""


text = example

In [ ]:
g = nx.DiGraph()

init, graph = text.split("\n\n")

val = {}
ops = {}

for l in init.splitlines():
    k, v = l.split(": ")
    val[k] = int(v)
    g.add_node(k)


for l in graph.splitlines():
    a, op, b, _, c = l.split()
    g.add_edge(a, c)
    g.add_edge(b, c)
    ops[c] = op

for n in nx.topological_sort(g):
    if n in val:
        continue

    p1, p2 = g.predecessors(n)
    op = ops[n]
    if op == "XOR":
        val[n] = val[p1] ^ val[p2]
    if op == "AND":
        val[n] = val[p1] & val[p2]
    if op == "OR":
        val[n] = val[p1] | val[p2]

zs = sorted([(k, v) for k, v in val.items() if k.startswith("z")], reverse=True)
print(zs)
s = int("".join(str(v) for k, v in zs), 2)


In [ ]:
s

In [ ]:
aocd.submit(s, day=24, year=2024, part="a")